<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Single-Record Ingestion for Multi-Label Multiclass Classification

In this tutorial, we'll outline how to send single-record prediction labels and actuals from multiclass models to Arize. Multiclass classification models are defined as a classification model with more than two classes. For a multiclass model, Multi-Label indicates the output of the model can be assigned to multiple classes. For more information on multiclass ingestion, please see our documentation <a href="https://docs.arize.com/arize/model-types/multiclass-classification">here</a>. For a full list of all model types, please see our documentation <a href="https://docs.arize.com/arize/">here</a>.

## Install and Import Dependencies

In [ ]:
!pip install -q arize
from arize.api import Client
from arize.utils.types import ModelTypes, Environments, MultiClassPredictionLabel, MultiClassActualLabel

import concurrent.futures as cf
import pandas as pd
import json
import urllib.request

## Download and Display Data
For this tutorial, we will use a sample JSON file representing a single prediction. 

In [ ]:
file_url = "https://storage.googleapis.com/arize-assets/documentation-sample-data/data-ingestion/multiclass-classification-assets/multi-label-multiclass-single-example.json"

with urllib.request.urlopen(file_url) as f:
    record = json.load(f)

print(json.dumps(record, indent=3))

## Create Arize Client
Sign up/login to your Arize account <a href="https://app.arize.com/auth/login">here</a>. Find your <a href="https://docs.arize.com/arize/api-reference/arize.log/client">Space and API keys</a>. Copy/paste into the cell below. 

In [ ]:
SPACE_KEY = "SPACE_KEY"  # update value here with your Space Key
API_KEY = "API_KEY"  # update value here with your API key

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print(
        "✅ Import and Setup Arize Client Done! Now we can start using Arize!"
    )

## Log Data to Arize
Log the record using the <a href="https://docs.arize.com/arize/sending-data-to-arize/data-ingestion-methods/sdk-reference/python-sdk/arize.log">single-record API</a>.
To log the record to Arize, you must pass in the <a href="https://docs.arize.com/arize/model-types/multiclass-classification#multi-class-prediction-and-threshold-values">MultiClassPredictionLabel</a> as the prediction_label parameter and the <a href="https://docs.arize.com/arize/model-types/multiclass-classification#multi-class-actual-values"</a> as the actual_label parameter in the log call.

In [ ]:
# log example to Arize
prediction_label = MultiClassPredictionLabel(
    prediction_scores=record["prediction_scores"],
    threshold_scores=record["threshold_scores"],
)
actual_label = MultiClassActualLabel(
    actual_scores=record["actual_scores"],
)
future = arize_client.log(
    model_id="multiclass-classification-multi-label-single-record-ingestion-tutorial",
    model_version="1.0",
    model_type=ModelTypes.MULTI_CLASS,
    environment=Environments.PRODUCTION,
    prediction_id=record["prediction_id"],
    prediction_label=prediction_label,
    actual_label=actual_label,
    features=record["features"]
)

res = future.result(timeout=10)
if res.status_code == 200:
    print(f"✅ future completed with response code {res.status_code}")
else:
    print(f"❌ future failed with response code {res.status_code}, {res.text}")